In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
cd /content/drive/MyDrive/Image-and-Video-Dehazing/

/content/drive/MyDrive/Image-and-Video-Dehazing


In [8]:
import cv2
import math
import numpy as np
import sys
from google.colab.patches import cv2_imshow

def apply_mask(matrix, mask, fill_value):
    #print("MATRIX=", matrix)
    #print("mask=\n" ,mask)
    #print("fill value=\n", fill_value)
                 
    masked = np.ma.array(matrix, mask=mask, fill_value=fill_value)
    #print('MASKED=',masked)
    return masked.filled()

def apply_threshold(matrix, low_value=255, high_value=255):
    low_mask = matrix < low_value
    #print("low mask=",low_mask)
    
    matrix = apply_mask(matrix, low_mask, low_value)
    #print('Low MASK->',low_mask,'\nMatrix->',matrix)

    high_mask = matrix > high_value
    matrix = apply_mask(matrix, high_mask, high_value)

    return matrix

def simplest_cb(img, percent):
    assert img.shape[2] == 3
    assert percent > 0 and percent < 100
    #print("shape of image = ", img.shape[2])

    half_percent = percent / 200.0
    #print('HALF PERCENT->',half_percent)

    channels = cv2.split(img)
    #print('Channels->\n',channels)
    #print('Shape->',channels[0].shape)
    #print('Shape of channels->',len(channels[2]))

    out_channels = []
    for channel in channels:
        assert len(channel.shape) == 2

	# find the low and high precentile values (based on the input percentile)
        height, width = channel.shape
        vec_size = width * height
        flat = channel.reshape(vec_size)
        #print('vec=',vec_size,'\nFlat=',flat)
        assert len(flat.shape) == 1

        flat = np.sort(flat)

        n_cols = flat.shape[0]
        #print("Number of columns = ", n_cols)

        low_val  = flat[math.floor(n_cols * half_percent)]
        high_val = flat[math.ceil( n_cols * (1.0 - half_percent))]

        #print("Lowval: ", low_val)
        #print("Highval: ", high_val)
        #print(flat[60])
        #print(flat[11940])
        

        # saturate below the low percentile and above the high percentile
        thresholded = apply_threshold(channel, low_val, high_val)
        # scale the channel
        normalized = cv2.normalize(thresholded, thresholded.copy(), 0, 255, cv2.NORM_MINMAX)
        out_channels.append(normalized)

    return cv2.merge(out_channels)

if __name__ == '__main__':
    #img = cv2.imread(sys.argv[1])
    cap=cv2.VideoCapture('/content/drive/MyDrive/Image-and-Video-Dehazing/haze-videos/dolphin.mp4')
    #img = cv2.imread('/home/dheeraj/Downloads/Whale.mov')
    
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    frame_width=640
    frame_height=480
    save = cv2.VideoWriter('/content/drive/MyDrive/Image-and-Video-Dehazing/hazy/output/output.mov',fourcc, 20.0, (frame_width,frame_height))
    
    #out = cv2.VideoWriter('output.avi',fourcc, 20.0, (1920,1080))
    while(cap.isOpened()):
        ret, frame=cap.read()
        if(ret== True):
            
            
            frame=cv2.resize(frame, (frame_width, frame_height))
            out = simplest_cb(frame, 1)
            cv2_imshow(frame) 
            cv2_imshow(out)
            out = cv2.flip(out,0)
            save.write(out)
            key=cv2.waitKey(1)
            if key==27:
                break
        else:
            break

    cap.release()
    save.release()
    cv2.destroyAllWindows()
    
    print("chalo bhai chal gyi video ")


Output hidden; open in https://colab.research.google.com to view.

In [10]:
!python dehaze.py

HALF PERCENT-> 0.005
Channels->
 (array([[31, 32, 34, ..., 52, 52, 51],
       [31, 32, 34, ..., 52, 52, 51],
       [31, 32, 32, ..., 52, 52, 51],
       ...,
       [ 7,  9, 10, ..., 12, 13, 15],
       [ 8, 10, 12, ..., 12, 11, 14],
       [ 6,  8, 10, ..., 12,  8, 12]], dtype=uint8), array([[165, 166, 169, ..., 213, 213, 212],
       [165, 167, 169, ..., 213, 213, 212],
       [166, 167, 169, ..., 213, 213, 212],
       ...,
       [143, 146, 147, ..., 162, 163, 163],
       [144, 147, 149, ..., 162, 161, 162],
       [144, 147, 150, ..., 161, 157, 159]], dtype=uint8), array([[21, 22, 23, ..., 33, 33, 32],
       [21, 21, 23, ..., 33, 33, 32],
       [20, 21, 23, ..., 33, 33, 32],
       ...,
       [55, 55, 55, ..., 56, 57, 57],
       [56, 56, 57, ..., 56, 55, 56],
       [56, 56, 57, ..., 57, 53, 55]], dtype=uint8))
Shape-> (389, 519)
Shape of channels-> 389
vec= 201891 
Flat= [31 32 34 ... 12  8 12]
Lowval:  7
Highval:  80
MASKED= [[31 32 34 ... 52 52 51]
 [31 32 34 ... 52 52 5

In [13]:
!python dehaze_video.py

Streaming output truncated to the last 5000 lines.
Flat= [204 204 204 ... 200 200 200]
Number of columns =  2073600
Lowval:  142
Highval:  231
108
144
low mask= [[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
MASKED= [[204 204 204 ... 214 214 214]
 [204 204 204 ... 213 213 213]
 [204 204 204 ... 211 211 211]
 ...
 [196 196 196 ... 200 200 200]
 [198 198 198 ... 200 200 200]
 [198 198 198 ... 200 200 200]]
Low MASK-> [[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]] 
Matrix-> [[204 204 204 ... 214 214 214]
 [204 204 204 ... 213 213 213]
 [204 204 204 ... 211 211 21

In [15]:
!python dehaze_multi.py

['/content/drive/MyDrive/Image-and-Video-Dehazing/hazy/1.jpg', '/content/drive/MyDrive/Image-and-Video-Dehazing/hazy/2a.jpg', '/content/drive/MyDrive/Image-and-Video-Dehazing/hazy/Dehazed', '/content/drive/MyDrive/Image-and-Video-Dehazing/hazy/UAE.jpg', '/content/drive/MyDrive/Image-and-Video-Dehazing/hazy/fog.jpg', '/content/drive/MyDrive/Image-and-Video-Dehazing/hazy/maxresdefault.jpg', '/content/drive/MyDrive/Image-and-Video-Dehazing/hazy/skydive.jpeg', '/content/drive/MyDrive/Image-and-Video-Dehazing/hazy/tort.jpg', '/content/drive/MyDrive/Image-and-Video-Dehazing/hazy/wood.jpg', '/content/drive/MyDrive/Image-and-Video-Dehazing/hazy/output', '/content/drive/MyDrive/Image-and-Video-Dehazing/hazy/.ipynb_checkpoints']
Directory= 



 ['', 'content', 'drive', 'MyDrive', 'Image-and-Video-Dehazing', 'hazy']
HALF PERCENT-> 0.005
Channels->
 (array([[179, 179, 179, ..., 118, 117, 117],
       [178, 178, 178, ..., 117, 116, 117],
       [176, 176, 176, ..., 118, 119, 119],
       ...,
     